<font size= "4">***CODE FOR FETCHING DATA FROM API FOR VOCATIONAL TRAININGS***</font>

In [25]:
import os
import requests
import jwt
import time
import sqlite3
import json
import re
from datetime import datetime , timedelta

def save_to_file(data, file_path):
    with open(file_path, 'w') as file:
        # file.write(data)
        json.dump(data, file)

def load_from_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def generate_jwt_token(client_id, client_secret):
    # Replace with the appropriate token endpoint URL for your OAuth server
    token_endpoint = "https://web.arbeitsagentur.de/ausbildungssuche/berufsausbildung-suche?sty=0&atyp=102&kat=0"

    now = int(time.time())
    payload = {
        "iss": client_id,
        "sub": client_id,
        "aud": token_endpoint,
        "iat": now,
        "exp": now + 3600,  # Token expires in 1 hour
    }

    # Sign the payload with your client secret to generate the JWT
    jwt_token = jwt.encode(payload, client_secret, algorithm='HS256')

    return jwt_token

def get_access_token(client_id, client_secret):
    # Generate the JWT token
    jwt_token = generate_jwt_token(client_id, client_secret)

    # Request the access token using the JWT token
    token_endpoint = "https://rest.arbeitsagentur.de/oauth/gettoken_cc"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    response = requests.post(token_endpoint, headers=headers, data=data)

    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to get access token. Status code: {response.status_code}")

def get_api_data(api_url, access_token):
    headers = {
        "Authorization": f"Bearer {access_token}",
    }

    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if data:  # Check if data is not empty
            return data
        else:
            print("No more data")
    else:
        raise Exception(f"Failed to fetch data from API. Status code: {response.status_code}")
    
def create_table(db_connection):
    cursor = db_connection.cursor()
    # Create a table if it doesn't exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS schulisch_ausbildung
                            (
                                veranstaltungs_ID INT PRIMARY KEY,
                                trainingstitel TEXT NULL,
                                dauer TEXT,
                                unterrichtsform TEXT,
                                abschlussart TEXT,
                                abschlussbezeichnung TEXT,
                                finanzielle_unterstützung  TEXT,
                                zugangsinformationen TEXT,
                                zielgruppe TEXT,
                                kredit TEXT,
                                anbieter_ID INT,
                                bildungsanbieter TEXT,
                                telefonVorwahl INT,
                                telefonDurchwahl INT,
                                mobilVorwahl INT,
                                mobilDurchwahl  INT,
                                faxVorwahl INT,
                                faxDurchwahl INT,
                                provider_homepage TEXT,
                                provider_email TEXT,
                                latitude REAL,
                                longitude REAL,
                                stadt TEXT,
                                land TEXT,
                                land_code TEXT ,
                                training_type TEXT,
                                schulart TEXT,
                                unterrichtszeiten TEXT,
                                kostenWert TEXT,
                                kostenWaehrung TEXT,
                                individuellerEinstieg TEXT,
                                foerderung TEXT,
                                link TEXT,
                                beginn DATE,
                                ende DATE,
                                teilnehmerMin INT,
                                teilnehmerMax INT,
                                aktualisierungsdatum DATE
                                )
                ''')
    db_connection.commit()

    
def insert_data_into_db(data, db_connection):
    #print(len(data))
    cursor = db_connection.cursor()
    for entry in data:
        
        veranstaltungs_ID = entry["id"]
        trainingstitel = entry["angebot"]["titel"]
        dauer = entry["dauer"]["bezeichnung"]
        unterrichtsform = entry["unterrichtsform"]["bezeichnung"]
        abschlussart = re.sub(r'<[^>]+>', '', str(entry["angebot"]["abschlussart"])) 
        abschlussbezeichnung = re.sub(r'<[^>]+>', '', str(entry["angebot"]["abschlussbezeichnung"])) 
        finanzielle_unterstützung = re.sub(r'<[^>]+>', '', str(entry["angebot"]["foerderung"])) 
        zugangsinformationen = re.sub(r'<[^>]+>', '', str(entry["angebot"]["zugang"])) 
        zielgruppe = re.sub(r'<[^>]+>', '', str(entry["angebot"]["zielgruppe"]))
        kredit = re.sub(r'<[^>]+>', '', str(entry["angebot"]["anrechnung"]))
        anbieter_ID = entry["angebot"]["bildungsanbieter"]["id"]
        bildungsanbieter = entry["angebot"]["bildungsanbieter"]["name"]
        telefonVorwahl = entry["angebot"]["bildungsanbieter"]["telefonVorwahl"]
        telefonDurchwahl = entry["angebot"]["bildungsanbieter"]["telefonDurchwahl"]
        mobilVorwahl = entry["angebot"]["bildungsanbieter"]["mobilVorwahl"]
        mobilDurchwahl  = entry["angebot"]["bildungsanbieter"]["mobilDurchwahl"]
        faxVorwahl = entry["angebot"]["bildungsanbieter"]["faxVorwahl"]
        faxDurchwahl = entry["angebot"]["bildungsanbieter"]["faxDurchwahl"]
        provider_homepage = entry["angebot"]["bildungsanbieter"]["homepage"]
        provider_email = entry["angebot"]["bildungsanbieter"]["email"]
        latitude = entry["angebot"]["bildungsanbieter"]["adresse"]["ortStrasse"]["koordinatenPlz"]["lat"]
        longitude = entry["angebot"]["bildungsanbieter"]["adresse"]["ortStrasse"]["koordinatenPlz"]["lon"]
        stadt = entry["angebot"]["bildungsanbieter"]["adresse"]["ortStrasse"]["name"]
        land = entry["angebot"]["bildungsanbieter"]["adresse"]["ortStrasse"]["land"]["name"]
        land_code = entry["angebot"]["bildungsanbieter"]["adresse"]["ortStrasse"]["land"]["code"]
        training_type = entry["angebot"]["bildungsart"]["bezeichnung"]
        schulart = entry["angebot"]["schulart"]["bezeichnung"]
        unterrichtszeiten = re.sub(r'<[^>]+>', '', str(entry["unterrichtszeiten"]))
        kostenWert = entry["kostenWert"]
        kostenWaehrung = entry["kostenWaehrung"]
        individuellerEinstieg = "ja" if entry["individuellerEinstieg"] == 1 else "nein"
        foerderung = "ja" if entry["foerderung"] == 1 else "nein"
        link = entry["link"]
        begin_timestamp = entry.get("beginn")
        if begin_timestamp is not None:
            beginn = (datetime.utcfromtimestamp(begin_timestamp / 1000)+ timedelta(days=1)).strftime('%Y-%m-%d')
        else:
            beginn = None

        end_timestamp = entry.get("ende")
        if end_timestamp is not None:
            ende = (datetime.utcfromtimestamp(end_timestamp / 1000)+ timedelta(days=1)).strftime('%Y-%m-%d')
        else:
            ende = None
        teilnehmerMin = entry["teilnehmerMin"] if entry["teilnehmerMin"] is not None else 0
        teilnehmerMax = entry["teilnehmerMax"] if entry["teilnehmerMax"] is not None else 0
        update_timestamp = entry["aktualisierungsdatum"]
        if update_timestamp is not None:
            aktualisierungsdatum = (datetime.utcfromtimestamp(update_timestamp / 1000)+ timedelta(days=1)).strftime('%Y-%m-%d')
        else:
            aktualisierungsdatum = None
        # Check if the event_id already exists in the table
        cursor.execute('''SELECT COUNT(*) FROM schulisch_ausbildung WHERE veranstaltungs_ID = ?''', (veranstaltungs_ID,))
        count = cursor.fetchone()[0]
        
        if count == 0: 
        # Insert data into the table
          cursor.execute('''INSERT INTO schulisch_ausbildung (
                                veranstaltungs_ID ,
                                trainingstitel ,
                                dauer ,
                                unterrichtsform ,
                                abschlussart ,
                                abschlussbezeichnung ,
                                finanzielle_unterstützung  ,
                                zugangsinformationen ,
                                zielgruppe ,
                                kredit ,
                                anbieter_ID ,
                                bildungsanbieter ,
                                telefonVorwahl ,
                                telefonDurchwahl ,
                                mobilVorwahl ,
                                mobilDurchwahl  ,
                                faxVorwahl ,
                                faxDurchwahl ,
                                provider_homepage ,
                                provider_email ,
                                latitude ,
                                longitude ,
                                stadt ,
                                land ,
                                land_code ,
                                training_type ,
                                schulart ,
                                unterrichtszeiten ,
                                kostenWert ,
                                kostenWaehrung ,
                                individuellerEinstieg ,
                                foerderung ,
                                link ,
                                beginn ,
                                ende ,
                                teilnehmerMin ,
                                teilnehmerMax ,
                                aktualisierungsdatum 
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                    veranstaltungs_ID ,
                    trainingstitel ,
                    dauer ,
                    unterrichtsform ,
                    abschlussart ,
                    abschlussbezeichnung ,
                    finanzielle_unterstützung  ,
                    zugangsinformationen ,
                    zielgruppe ,
                    kredit ,
                    anbieter_ID ,
                    bildungsanbieter ,
                    telefonVorwahl ,
                    telefonDurchwahl ,
                    mobilVorwahl ,
                    mobilDurchwahl  ,
                    faxVorwahl ,
                    faxDurchwahl ,
                    provider_homepage ,
                    provider_email ,
                    latitude ,
                    longitude ,
                    stadt ,
                    land ,
                    land_code,
                    training_type  ,
                    schulart ,
                    unterrichtszeiten ,
                    kostenWert ,
                    kostenWaehrung ,
                    individuellerEinstieg ,
                    foerderung ,
                    link ,
                    beginn ,
                    ende ,
                    teilnehmerMin ,
                    teilnehmerMax ,
                    aktualisierungsdatum
            ))
    db_connection.commit()
    #print("insert data")
    
def main():
    client_id = "1c852184-1944-4a9e-a093-5cc078981294"  # Replace with your OAuth client ID
    client_secret = "777f9915-9f0d-4982-9c33-07b5810a3e79"  # Replace with your OAuth client secret

    # Get the access token using client credentials flow with JWTs
    access_token = get_access_token(client_id, client_secret)

    # List of values for the 'dauer' parameter
    dauer_values = ['0','1','2','3','4','5','6','7','8','9']


    try:
        # Open a connection
        db_connection = sqlite3.connect('weiterbildung_analysis.db')

        # Create the table if not exists
        create_table(db_connection)

        for dauer_value in dauer_values:
            api_url = f"https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&dauer={dauer_value}"
            page = 0
            total_pages = 1

            # Pagination: Fetch all data from the API using multiple requests
            all_data = []

            while page < total_pages:
                paginated_api_url = f"{api_url}&page={page}&size=10000"  # Updated pagination URL
                print(paginated_api_url)
                api_data = get_api_data(paginated_api_url, access_token)

                if "_embedded" in api_data:
                    all_data.extend(api_data["_embedded"]["termine"])  # Append to the existing data
                else:
                    print("No more data")
                    break

                # Update total_pages based on the response
                total_pages = api_data["page"]["totalPages"]
                page += 1

            # Insert data into the database
            insert_data_into_db(all_data, db_connection)

    except Exception as e:
        print(f"Error: {e}")
        db_connection.rollback()  # Rollback changes in case of an error
    
    finally:
        # Close the connection
        db_connection.close()
        print("Data fetched and saved to SQLite database successfully.")


if __name__ == "__main__":
    main()


https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&dauer=0&page=0&size=10000
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&dauer=1&page=0&size=10000
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&dauer=2&page=0&size=10000
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&dauer=3&page=0&size=10000
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&dauer=4&page=0&size=10000
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&dauer=4&page=1&size=10000
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&dauer=5&page=0&size=10000
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&dauer=5&page=1&size=10000
https://rest.arbeitsagentur.de/infosysbub/absuche/pc/v1/ausbildungsangebot?bart=102&dauer=6&page=0&size=10000
https://re

<font size= "4">***PIE CHART FOR SHOWING THE DISTRIBUTION OF DAUER ACROSS TRAININGS***</font>

In [26]:
import sqlite3
import plotly.express as px

db_connection = sqlite3.connect('weiterbildung_analysis.db')
cursor = db_connection.cursor()

query = '''
    SELECT dauer, COUNT(*) as count
    FROM schulisch_ausbildung
    GROUP BY dauer
    ORDER BY dauer
'''
cursor.execute(query)
results = cursor.fetchall()

db_connection.close()

dauer_values = [row[0] for row in results]
counts = [row[1] for row in results]

total_counts = sum(counts)
percentages = [(count / total_counts) * 100 for count in counts]

fig = px.pie(
    values=percentages,
    names=dauer_values,
    title='Distribution of Dauer Across Training Courses (Percentage)',
    labels={'dauer_values': 'Dauer'}  # Display dauer_values outside the chart
)
fig.update_traces(textinfo='percent')

# Center the title above the chart
fig.update_layout(title_x=0.5)

fig.show()


<font size= "4">***BAR CHART FOR SHOWING THE TOP 20 STATES THAT OFFER MOST TRAININGS***</font>

In [40]:
import sqlite3
import plotly.express as px

db_connection = sqlite3.connect('weiterbildung_analysis.db')
cursor = db_connection.cursor()

query = '''
    SELECT land, COUNT(*) as count
    FROM schulisch_ausbildung
    WHERE land IS NOT NULL
    GROUP BY land
    ORDER BY count DESC
    LIMIT 20
'''
cursor.execute(query)
data = cursor.fetchall()

db_connection.close()

state_names = [entry[0] for entry in data]
training_counts = [entry[1] for entry in data]

fig = px.bar(
    x=state_names,
    y=training_counts, # Color bars based on state names
    title='German States with Most Training Offerings',
    labels={'x': 'German States', 'y': 'Number of Trainings'},
)

# Display the number of trainings on top of each bar
fig.update_traces(texttemplate='%{y}', textposition='outside')

# Rotate x-axis labels for better readability
fig.update_layout(xaxis_tickangle=-45)

# Set y-axis range to accommodate larger values
yaxis_range = [0, max(training_counts) + 1000]
fig.update_layout(yaxis_range=yaxis_range,title_x=0.5)

fig.show()


<font size= "4">***BAR CHART FOR SHOWING THE DISTRIBUTION OF TRAININGS WITH FOERDERUNG***</font>

In [41]:
import sqlite3
import plotly.graph_objects as go

db_connection = sqlite3.connect('weiterbildung_analysis.db')
cursor = db_connection.cursor()

query_ja = "SELECT COUNT(*) FROM schulisch_ausbildung WHERE foerderung = 'ja'"
cursor.execute(query_ja)
count_ja = cursor.fetchone()[0]

query_nein = "SELECT COUNT(*) FROM schulisch_ausbildung WHERE foerderung = 'nein'"
cursor.execute(query_nein)
count_nein = cursor.fetchone()[0]

db_connection.close()

categories = ['ja', 'nein']
counts = [count_ja, count_nein]

fig = go.Figure(data=[
    go.Bar(x=categories, y=counts, marker_color=['darkcyan', 'lightseagreen'])
])

# Add labels to the bars
for count in counts:
    fig.add_annotation(
        text=str(count),
        x=categories[counts.index(count)],
        y=count,
        showarrow=True,
        font=dict(size=10)
    )

fig.update_layout(
    xaxis_title='foerderung',
    yaxis_title='Number of Trainings',
    title='Number of Trainings with foerderung',
    title_x=0.5
)

fig.show()


<font size= "4">***DOUGHNUT PIE CHART FOR SHOWING THE DISTRIBUTION OF TRAININGS WITH FOERDERUNG***</font>

In [58]:
import sqlite3
import plotly.graph_objects as go

db_connection = sqlite3.connect('weiterbildung_analysis.db')
cursor = db_connection.cursor()

query_ja = "SELECT COUNT(*) FROM schulisch_ausbildung WHERE foerderung = 'ja'"
cursor.execute(query_ja)
count_ja = cursor.fetchone()[0]

query_nein = "SELECT COUNT(*) FROM schulisch_ausbildung WHERE foerderung = 'nein'"
cursor.execute(query_nein)
count_nein = cursor.fetchone()[0]

db_connection.close()

labels = ['ja', 'nein']
values = [count_ja, count_nein]

fig = go.Figure(data=[
    go.Pie(labels=labels, values=values, hole=0.4)
])

fig.update_layout(
    title='Distribution of Trainings with foerderung',
    title_x=0.5 
)

fig.show()


<font size= "4">***BAR CHART FOR SHOWING TOP PROVIDERS WITH MOST TRAININGS***</font>

In [59]:
import sqlite3
import plotly.express as px

db_connection = sqlite3.connect('weiterbildung_analysis.db')
cursor = db_connection.cursor()

query = '''
    SELECT bildungsanbieter, COUNT(*) as count
    FROM schulisch_ausbildung
    WHERE bildungsanbieter IS NOT NULL
    GROUP BY bildungsanbieter
    ORDER BY count DESC
    LIMIT 10
'''
cursor.execute(query)
data = cursor.fetchall()

db_connection.close()

provider_names = [entry[0] for entry in data]
training_counts = [entry[1] for entry in data]

fig = px.bar(
    x=provider_names,
    y=training_counts,
    title='Top Providers with Most Trainings',
    labels={'x': 'Providers', 'y': 'Number of Trainings'},
    text=training_counts  
)

fig.update_xaxes(tickangle=45, tickfont=dict(size=10))

fig.update_traces(texttemplate='%{text}', textposition='outside')

fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5))

yaxis_range = [0, max(training_counts) + 3000]  
fig.update_yaxes(range=yaxis_range)

fig.show()


<font size= "4">***BAR CHART FOR SHOWING TRAINING COUNT BY REGION***</font>

In [51]:
import sqlite3
import plotly.express as px

db_connection = sqlite3.connect('weiterbildung_analysis.db')
cursor = db_connection.cursor()

query = '''
    SELECT land_code, COUNT(*) as count
    FROM schulisch_ausbildung
    WHERE land_code IS NOT NULL
    GROUP BY land_code
    ORDER BY land_code ASC
'''
cursor.execute(query)
data = cursor.fetchall()

db_connection.close()

region_codes = [entry[0] for entry in data]
training_counts = [entry[1] for entry in data]

fig = px.bar(
    x=region_codes,
    y=training_counts,
    title='Training Count by Region (Land Code)',
    labels={'x': 'Region (Land Code)', 'y': 'Number of Trainings'},
    text=training_counts 
)

fig.update_xaxes(tickangle=45, tickfont=dict(size=10))

fig.update_traces(texttemplate='%{text}', textposition='outside')

fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5))

fig.show()


<font size= "4">***BAR CHART FOR SHOWING TRAINING COUNT BY REGION AND INSTRUCTIONAL FORMAT***</font>

In [52]:
import sqlite3
import plotly.express as px

db_connection = sqlite3.connect('weiterbildung_analysis.db')
cursor = db_connection.cursor()

query = '''
    SELECT land, unterrichtsform, COUNT(*) as count
    FROM schulisch_ausbildung
    WHERE land IS NOT NULL AND unterrichtsform IS NOT NULL
    GROUP BY land, unterrichtsform
    ORDER BY count DESC
    limit 20
'''
cursor.execute(query)
data = cursor.fetchall()

db_connection.close()

import pandas as pd
df = pd.DataFrame(data, columns=['land', 'unterrichtsform', 'count'])

fig = px.bar(
    df,
    x='land',
    y='count',
    color='unterrichtsform',
    title='Training Count by Region (Land) and Instructional Format',
    labels={'x': 'Region (Land)', 'y': 'Number of Trainings'},
    text='count',  
    barmode='group'  
)

# Rotate x-axis labels for better visibility
fig.update_xaxes(tickangle=45, tickfont=dict(size=10))

# Display the number of trainings on top of each bar
fig.update_traces(texttemplate='%{text}', textposition='outside')

# Set the position of the legend to be below the chart
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5))

# Show the plot
fig.show()


<font size= "4">***PIE CHART FOR SHOWING DISTRIBUTION OF INSTRUCTIONAL FORMATS***</font>

In [53]:
import sqlite3
import plotly.express as px

db_connection = sqlite3.connect('weiterbildung_analysis.db')
cursor = db_connection.cursor()

query = '''
    SELECT unterrichtsform, COUNT(*) as count
    FROM schulisch_ausbildung
    WHERE unterrichtsform IS NOT NULL
    GROUP BY unterrichtsform
    ORDER BY count DESC
'''
cursor.execute(query)
data = cursor.fetchall()

db_connection.close()

import pandas as pd
df = pd.DataFrame(data, columns=['unterrichtsform', 'count'])

total_count = df['count'].sum()

df['percentage'] = (df['count'] / total_count) * 100

fig = px.pie(
    df,
    names='unterrichtsform',
    values='percentage',
    title='Distribution of Instructional Formats',
    labels={'unterrichtsform': 'Instructional Format'}
)

fig.update_layout(title_x=0.5)
fig.show()


<font size= "4">***MAP FOR SHOWING TRAINING PROVIDERS WITH THE NUMBER OF TRAININGS BY CITY***</font>

In [65]:
import pandas as pd
import geopandas as gpd
import folium
import sqlite3

# Read the GeoJSON file containing the geometries of German states
path = "NUTS_RG_20M_2021_3035.json"
gdf = gpd.read_file(path)

# Specify CRS (see the filename)
gdf.crs = "EPSG:3035"

# Filter and transform the data for visualization
gdf_de = gdf[gdf.CNTR_CODE == "DE"]
gdf_de = gdf_de.to_crs("EPSG:4326")  # Change CRS for Folium compatibility

# Creating a Folium map centered on Germany
m = folium.Map(location=[51.1657, 10.4515], zoom_start=6, control_scale=True)

db_connection = sqlite3.connect('weiterbildung_analysis.db')
cursor = db_connection.cursor()

query = '''
    SELECT stadt, COUNT(*) as training_count, latitude, longitude, bildungsanbieter
    FROM schulisch_ausbildung
    WHERE stadt IS NOT NULL
    GROUP BY stadt
    ORDER BY training_count DESC
'''
cursor.execute(query)
data = cursor.fetchall()

db_connection.close()

city_trainings_df = pd.DataFrame(data, columns=['stadt', 'training_count', 'latitude', 'longitude', 'bildungsanbieter'])

for idx, row in city_trainings_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['training_count'] / 10, 
        popup=f"{row['stadt']} ({row['bildungsanbieter']}): {row['training_count']} trainings",
        color='red',
        fill=True,
        fill_color='red'
    ).add_to(m)

# Saving the map in the same folder
map_filename = "Schulisch_ausbildung_Map_showing_the_training_providers_with_the_number_of_trainings_by_city.html"
m.save(map_filename)

print(f"Map with training providers has been saved as '{map_filename}' in your folder.")


Map with training providers has been saved as 'Schulisch_ausbildung_Map_showing_the_training_providers_with_the_number_of_trainings_by_city.html' in your folder.


In [66]:
import pandas as pd
import geopandas as gpd
import folium
import sqlite3

# Read the GeoJSON file containing the geometries of German states
path = "NUTS_RG_20M_2021_3035.json"
gdf = gpd.read_file(path)

# Specify CRS (see the filename)
gdf.crs = "EPSG:3035"

# Filter and transform the data for visualization
gdf_de = gdf[gdf.CNTR_CODE == "DE"]
gdf_de = gdf_de.to_crs("EPSG:4326")  # Change CRS for Folium compatibility

# Creating a Folium map centered on Germany with a darker background (CartoDB Dark_Matter)
m = folium.Map(location=[51.1657, 10.4515], zoom_start=6, control_scale=True, tiles='CartoDB Dark_Matter')

db_connection = sqlite3.connect('weiterbildung_analysis.db')
cursor = db_connection.cursor()

query = '''
    SELECT stadt, COUNT(*) as training_count, latitude, longitude, bildungsanbieter
    FROM schulisch_ausbildung
    WHERE stadt IS NOT NULL
    GROUP BY stadt
    ORDER BY training_count DESC
    LIMIT 20
'''
cursor.execute(query)
data = cursor.fetchall()

db_connection.close()

city_trainings_df = pd.DataFrame(data, columns=['stadt', 'training_count', 'latitude', 'longitude', 'bildungsanbieter'])

# Define a color scheme for cities based on training counts
colors = ['red', 'blue', 'green', 'purple', 'orange', 'pink', 'gray', 'darkred', 'lightgreen', 'lightblue']

for idx, row in city_trainings_df.iterrows():
    # Assign a color based on the rank of training counts (up to 10 unique colors)
    color = colors[idx % len(colors)]
    
    # Create a formatted popup with the city name highlighted, dynamic width, and increased font size
    city_name = row['stadt']
    provider_name = row['bildungsanbieter']
    training_count = row['training_count']
    
    # Calculate the width based on the length of the content
    popup_width = max(len(city_name), len(provider_name), len(str(training_count))) * 10 + 30
    
    # Increase the font size using CSS styling
    popup_html = f"<div style='width: {popup_width}px; font-size: 14px;'><strong>{city_name}</strong><br>({provider_name}): {training_count} trainings</div>"
    
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_html, max_width=popup_width),
        icon=folium.Icon(color=color)
    ).add_to(m)

# Saving the map in the same folder
map_filename = "Schulisch_ausbildung_Map_showing_top_20_providers_with_number_of_training_by_city.html"
m.save(map_filename)

print(f"Map for showing top 20 vocational trainings by city has been saved as '{map_filename}' in your folder.")

Map for showing top 20 vocational trainings by city has been saved as 'Schulisch_ausbildung_Map_showing_top_20_providers_with_number_of_training_by_city.html' in your folder.
